In [10]:
from tdmclient import ClientAsync, aw
import math as m
import numpy as np

import importlib, Control_fromEKF
import Control_fromEKF
importlib.reload(Control_fromEKF)
from Control_fromEKF import follow_astar_path_ekf, get_cmd

import importlib, Filtering
import Filtering
importlib.reload(Filtering)
from Filtering import EKFState, ekf_task, get_motor_meas
# params
Ts = 0.1  # time step in seconds
L = 95  # distance between wheels in mm
speed_to_mms = 0.3375  # conversion factor from thymio speed units to mm/s from solution ex.8 (in our measurement it was 0.43478260869565216)

# Process noise for EKF (tune) (from model-mismatch/random-walk/control execution)
q_proc = (
    1e-3, 1e-2, 1e-4,   # q_x, q_y, q_theta (model mismatch)
    75.72,  0.002692,         # q_v_ctrl, q_omega_ctrl (control execution noise)
    1e-2, 1e-5          # q_v_bias, q_omega_bias (random walk on v, omega)
)
# Camera measurement noise (tune)
r_cam = (1.435, 1.864, 0.001496)  # [mm^2, mm^2, rad^2]
r_mot = (75.72, 0.002692)    # motor noise on v, omega


# Suppose elsewhere you update ekf_state = (x_mm, y_mm, th_rad) at ~1/dt Hz
ekf = EKFState(x0=[0,0,0,0,0], P0=1000*np.eye(5))
def ekf_get_state():
    return ekf.ekf_state  # (x,y,theta)

client = ClientAsync()
node = aw(client.wait_for_node())
try:
    aw(node.lock()) # lock the node for R/W
except Exception:
    pass # ignore it it wasn't locked

aw(node.stop())
print("Connected:", node)
print("i wait for variables")
aw(node.wait_for_variables({"motors.left.speed","motors.right.speed"}))
print("i got around wait for variables, yay")
aw(ekf_task(ekf, node, client, Ts=Ts, speed_to_mms=speed_to_mms, L=L, q_proc=q_proc,
    r_cam=r_cam, r_mot=r_mot, get_cam_meas=None, get_motor_meas=get_motor_meas(node), get_cmd=get_cmd,
))

try:
    path = [(0,0), (1,0), (2,2), (3,2), (4,3)]  # grid cells
    x, y, th = aw(follow_astar_path_ekf(
        node=node, client=client, ekf_get_state=ekf_get_state,
        path_cells=path, cell_size_mm=20.0,
        use_smooth_vw=True,       # try smooth V–ω once EKF is stable
        v_cmd=120, kp_heading=220.0,
        dt=0.02, pos_tol=10.0, ang_tol=0.03
    ))
    print(f"Final pose (EKF): x={x:.1f} mm, y={y:.1f} mm, th={m.degrees(th):.1f} deg")
finally:
    aw(node.set_variables({"motor.left.target":[0], "motor.right.target":[0]}))
    aw(node.unlock())

Connected: Node 30f587b4-fb96-452d-aa4f-57791877b058
i wait for variables


KeyboardInterrupt: 